In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor

In [2]:
order_data = pd.DataFrame()
for i in range(1, 22):
    order_data = order_data.append(pd.read_csv("./training_data/order_data/order_data_2016-01-%02d" % i, header = None, delim_whitespace = True))
cluster_map = pd.read_csv("./training_data/cluster_map/cluster_map", header = None, delim_whitespace = True)

In [3]:
order_data.head(10)

,0,1,2,3,4,5,6,7
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01,13:37:23
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01,09:47:54
2,abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01,18:24:02
3,cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,2016-01-01,22:13:27
4,139d492189ae5a933122c098f63252b3,NaN,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,2016-01-01,17:00:06
5,b0b59fd0fe98bf603972da2f62e6522d,NaN,970fc2d834c16362cfed666e4baad9f8,d4ec2125aff74eded207d2d915ef682f,8bb37d24db1ad665e706c2655d9c4c72,44.0,2016-01-01,06:10:34
6,17c1c85144ab532947c7ea724fdcc945,115ac9b23f00a2e6d8a3041e23469f41,2f206d28eb6d7daa6d058304c00782de,a5609739c6b5c2719a3752327c5e33a7,a5609739c6b5c2719a3752327c5e33a7,6.0,2016-01-01,17:34:33
7,d682c1c004024f8937d21cd43498d1bb,7cb7ccbc192c177b136feca274fb3d07,9cc0a783df3ea8ae3067ba2264d87fc3,74c1c25f4b283fa74a5514307b0d0278,74c1c25f4b283fa74a5514307b0d0278,3.0,2016-01-01,12:12:55
8,6fcae38baf2eb52e17273df41bf6fc6f,c586e492c4587c3aa15557a8eb41806d,c706cc35b05d84b224dec9d429c11de0,4725c39a5e5f4c188d382da3910b3f3f,4725c39a5e5f4c188d382da3910b3f3f,5.0,2016-01-01,14:28:07
9,70afc52be8a6d35137f2277a6ca88017,ea61e970ba415f946b3b35464c74f32a,cef9c611fab48a823b02d742c89207a9,87285a66236346350541b8815c5fae94,52a4e8aaa12f70020e889aed8fd5ddbc,5.0,2016-01-01,21:56:29


In [4]:
order_data["time_slice"] = order_data[7].apply(lambda x: (int(x.split(':')[0]) * 60 * 60 + int(x.split(':')[1]) * 60 + int(x.split(':')[2])) / 600)

In [5]:
order_data['time_slice'].max()

143

In [6]:
region_mapping = {}
for item in cluster_map.values:
    region_mapping[item[0]] = item[1]

In [7]:
order_data["region_id"] = order_data[3].apply(lambda x: int(region_mapping[x]))

In [8]:
order_data["date"] = order_data[6].apply(lambda x: int(x.split('-')[2]) - 1)

In [9]:
print order_data['time_slice'].min()
print order_data['time_slice'].max()
print order_data['region_id'].min()
print order_data['region_id'].max()
print order_data['date'].min()
print order_data['date'].max()

0
143
1
66
0
20


In [10]:
demand_count = {}
supply_count = {}
for i in range(0, 21):
    for j in range(0, 144):
        demand_count[i * 144 + j] = {}
        supply_count[i * 144 + j] = {}
        for z in range(1, 67):
            demand_count[i * 144 + j][z] = 0
            supply_count[i * 144 + j][z] = 0

In [11]:
order_data.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, u'time_slice', u'region_id', u'date'], dtype='object')

In [12]:
for item in order_data.values:
    demand_count[item[10] * 144 + item[8]][item[9]] += 1
    if pd.isnull(item[1]) == False:
        supply_count[item[10] * 144 + item[8]][item[9]] += 1

In [21]:
data = np.array([[0, 0, 0, 0, 0, 0, 0]])

In [22]:
for i in range(1, 67):
    for j in range(3, 144 * 21):
        tsi = j % 144
        h = j / 144 + 1
        if h == 1 or h == 2 or h == 3 or h == 9 or h == 10 or h == 16 or h == 17:
            h = 1;
        else:
            h = 0;
        data = np.vstack([data, [h, i, tsi, abs(demand_count[j - 1][i] - supply_count[j - 1][i]), abs(demand_count[j - 2][i] - supply_count[j - 2][i]), abs(demand_count[j - 3][i] - supply_count[j - 3][i]), abs(demand_count[j][i] - supply_count[j][i])]])

In [23]:
training_data = pd.DataFrame(data[1:], columns = ["is_holiday", "region_id", "time_slice_id", "gap_1", "gap_2", "gap_3", "gap"])

In [26]:
training_data["is_holiday"].unique()

array([1, 0], dtype=int64)

In [27]:
target = training_data["gap"]
train = training_data.drop("gap", 1)

In [28]:
print train.columns

Index([u'is_holiday', u'region_id', u'time_slice_id', u'gap_1', u'gap_2',
       u'gap_3'],
      dtype='object')


In [29]:
from sklearn.cross_validation import KFold
def loss(act, pred):
    act = act[act > 0]
    pred = pred[act > 0]
    l = abs(act - pred) / act
    return sum(l) / len(l)

In [31]:
best_n = 0
best_d = 0
best_score = 0
for n in (10, 17):
    for d in (None, 20):
        rfc = RandomForestRegressor(n_estimators=n, max_depth=d, n_jobs=-1)
        rfc_score = 0;
        for train_k, test_k in KFold(len(train), n_folds=10, shuffle=True):
            rfc.fit(train.iloc[train_k], target.iloc[train_k])
            pred = rfc.predict(train.iloc[test_k])
            #rfc_score += rfc.score(train.iloc[test_k], target.iloc[test_k]) / 10.0
            rfc_score += loss(target.iloc[test_k], pred) / 10.0
        print("%s %s %s" %(n, d, rfc_score))
        if (rfc_score > best_score):
            best_score = rfc_score
            best_n = n
            best_d = d

C:\Users\shuan\AppData\Local\Continuum\Miniconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


10 None 11.7477825588
10 20 12.6298599784
17 None 2.09070090321
17 20 11.8252659614


In [108]:
print best_d, best_n, best_score

20 17 0.807406834598


In [109]:
rfc = RandomForestRegressor(n_estimators=17, max_depth=18)
rfc.fit(train, target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=18,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=17, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [110]:
rfc.score(train, target)

0.96666478812545187

In [111]:
test = pd.read_csv("test.csv", header=None)

In [112]:
result = test.copy()

In [113]:
result.head()

,0,1,2
0,1,2016-01-22-46,0
1,2,2016-01-22-46,0
2,3,2016-01-22-46,0
3,4,2016-01-22-46,0
4,5,2016-01-22-46,0


In [114]:
test_order_data = pd.DataFrame()
for i in [22, 24, 26, 28, 30]:
    test_order_data = test_order_data.append(pd.read_csv("./test_set_1/order_data/order_data_2016-01-%02d_test" % i, header=None, delim_whitespace=True))

In [115]:
test_order_data.head()

,0,1,2,3,4,5,6,7
0,e37f842c2a37de68e16466a3c56b916b,NaN,5903295e07afb893e1a0f7073965515d,b05379ac3f9b7d99370d443cfd5dcc28,82cc4851f9e4faa4e54309f8bb73fd7c,7.0,2016-01-22,17:19:43
1,62588e55ff8892ba38a0bbe5678be272,53919c82b12bd39b12d77d4d8db1dda1,5903295e07afb893e1a0f7073965515d,b05379ac3f9b7d99370d443cfd5dcc28,82cc4851f9e4faa4e54309f8bb73fd7c,12.0,2016-01-22,17:19:43
2,693194e4d57cdd500e793c0c1e4f7a93,19f5ba02d33855688b727e39c98b2939,dbea2d56b9fcacf8b18c40a4f34fbbf2,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,18.0,2016-01-22,11:00:05
3,3a0c651d2558d9083a66fc179e3ba81c,NaN,4b9957e8fb4d8cf981560e2a512aa598,d4ec2125aff74eded207d2d915ef682f,d1ab2cc538d518758a1a82b1787592d4,36.0,2016-01-22,17:00:42
4,68b7cec210c7f875b79ce32dab7195ea,NaN,4b9957e8fb4d8cf981560e2a512aa598,d4ec2125aff74eded207d2d915ef682f,d1ab2cc538d518758a1a82b1787592d4,36.0,2016-01-22,17:00:42


In [116]:
test_order_data["time_slice"] = test_order_data[7].apply(lambda x: (int(x.split(':')[0]) * 60 * 60 + int(x.split(':')[1]) * 60 + int(x.split(':')[2])) / 600)

In [117]:
test_order_data["region_id"] = test_order_data[3].apply(lambda x: int(region_mapping[x]))

In [118]:
test_order_data["date"] = test_order_data[6].apply(lambda x: (int(x.split('-')[2]) - 22) / 2)

In [119]:
test_order_data["time_slice"].unique()

array([103,  66, 102, 126, 104,  55, 139, 116,  44, 127, 138,  56,  92,
        80, 115,  43, 128, 114,  68,  67,  42,  91,  54,  90,  78, 140,  79])

In [120]:
test_demand = {}
test_supply = {}
for i in range(0, 5):
    for j in range(0, 144):
        test_demand[i * 144 + j] = {}
        test_supply[i * 144 + j] = {}
        for z in range(1, 67):
            test_demand[i * 144 + j][z] = 0
            test_supply[i * 144 + j][z] = 0

In [121]:
for item in test_order_data.values:
    test_demand[item[10] * 144 + item[8]][item[9]] += 1
    if pd.isnull(item[1]) == False:
        test_supply[item[10] * 144 + item[8]][item[9]] += 1

In [122]:
result[1] = result[1].apply(lambda x : (int(x.split('-')[2]) - 22) / 2 * 144 + int(x.split('-')[3]))

In [123]:
result[1].unique()

array([ 46,  58,  70,  82,  94, 106, 118, 130, 142, 202, 214, 226, 238,
       250, 262, 274, 286, 334, 346, 358, 370, 382, 394, 406, 418, 430,
       490, 502, 514, 526, 538, 550, 562, 574, 622, 634, 646, 658, 670,
       682, 694, 706, 718])

In [124]:
test_data = np.array([[0, 0, 0, 0, 0]])

In [125]:
for item in result.values:
    test_data = np.vstack([test_data, [item[0], item[1] % 144, abs(test_demand[item[1] - 2][item[0]] - test_supply[item[1] - 2][item[0]]), abs(test_demand[item[1] - 3][item[0]] - test_supply[item[1] - 3][item[0]]), abs(test_demand[item[1] - 4][item[0]] - test_supply[item[1] - 4][item[0]])]])

In [126]:
test_data.shape

(2839, 5)

In [127]:
test_data_frame = pd.DataFrame(test_data[1:], columns = ["region_id", "time_slice_id", "gap_1", "gap_2", "gap_3"])

In [128]:
test_data_frame['gap_2'].unique()

array([  1,   2,   0,   8,  46,  17,   7,  11,  15,   3,   9,   5,  96,
        47, 198,   4,  16, 132, 123,  37, 110, 389,  32, 197,   6,  14,
        21,  27,  50,  61,  13,  33,  19, 129,  28,  68,  77,  20, 116,
        10,  94,  70,  40,  12,  57,  43, 391, 227,  55,  34, 130,  23,
       124, 230, 608,  58,  38, 111,  18,  71,  85, 255,  22, 150, 109,
       127, 619,  42,  41, 245,  81,  76, 102,  64,  83, 141, 155,  24,
        36,  30, 140,  31,  88, 661, 184,  39,  86,  44, 412,  62, 238,
       131, 103, 418, 208,  26,  95, 219,  60,  49,  51,  87, 108, 164,
       216,  63,  67,  99, 153, 169,  52, 182, 405, 310, 119,  59, 101,
       490, 924,  73,  25,  29,  97, 167, 202,  80, 962, 280,  35, 263,
       125,  72, 171,  56,  66,  65, 137, 183, 106, 368, 236, 113, 210,
       374, 823, 112, 204, 135, 159, 365, 207, 187,  69,  45, 158, 120,
       267, 411, 222,  78, 880, 265])

In [129]:
test_data_frame["gap_3"].unique()

array([   4,    1,    0,    2,    3,   14,    5,   11,    6,   79,   20,
        154,    8,    9,    7,   74,   77,   29,   58,   21,  242,  139,
         10,   17,   13,   48,   57,   15,   18,   16,   44,  141,   22,
         41,  131,   28,   12,  164,  107,   24,   56,   19,   51,  129,
        725,  417,   54,   31,  160,   52,  144,  477, 1164,   62,   37,
         26,  111,   27,  126,  245,   33,   32,  161,  176,   25,  187,
        624,   34,  282,   36,  128,  133,  149,   43,   90,  660,   23,
         30,  104,   53,  177,  153,  100,  115,  158,   49,   39,   35,
         50,   93,   59,  159,   80,   63,  120,   71,  116,  182,  289,
        605,  202,   60,   40,   91,  583,  933,  113,   42,  865,   70,
        252,   68,  194,   92,   55,  134,  169,  124,  132,   38,   98,
         76,  211,   82,   46,  135,  457,  991,   83,  130,   96,  240,
        201,  228,  121,  662,  207,   67,   69,  119,  106,   84,  276,
        213,   89,   64,  190,  125,   81,  181,  2

In [130]:
test[2] = rfc.predict(test_data_frame)

In [131]:
test.to_csv("predict_submission3.csv", index=False, header=None)